In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import myML
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [ ]:
def RGBtohue(RGB):
    R, G, B = RGB
    if R>=G and G>=B: temp = (G-B)/(R-B)
    elif G>=R and R>=B: temp = 2-(R-B)/(G-B)
    elif G>=B and B>=R: temp = 2+(B-R)/(G-R)
    elif B>=G and G>=R: temp = 4-(G-R)/(B-R)
    elif B>=R and R>=G: temp = 4+(R-G)/(B-G)
    elif R>=B and B>=G: temp = 6-(B-G)/(R-G)
    return temp*60

In [ ]:
# Load training images. Each image shouldn't have unwanted pixels (surroundings) and must be named with the known pH value
location = os.getcwd()+'\\training\\'
rg = np.empty((2,len(os.listdir(location))))
Y_train = np.empty((1,len(os.listdir(location))))
X_train = np.empty((1,len(os.listdir(location))))

for count, file in enumerate(os.listdir(location)):
    RGB = [np.mean(x) for x in np.array(Image.open(location+file)).T]
    X_train[0,count] = RGBtohue(RGB)+20
    if X_train[0,count]>360: X_train[0,count]-=360
    Y_train[0,count] = float(file.split('.jpg')[0])

#Sort by ascending ph
order = np.argsort(X_train)[0]
X_train = X_train[:,order]
Y_train = Y_train[:,order]


plt.hlines(1,0,360)  # Draw a horizontal line
plt.xlim(0,360)
plt.ylim(0.5,1.5)

y = np.ones(np.shape(X_train[0,:]))   # Make all y values the same
plt.plot(X_train[0,:],y,'|',ms = 40)  # Plot a line at each location specified
for count in range(X_train.shape[1]):
    if count % 2: pos = 1.2
    else: pos = 0.8
    plt.annotate(round(Y_train[0,count],1), [X_train[0,count], pos], fontsize=8)
plt.axis('off')
plt.rcParams['figure.figsize'] = (24, 3)
plt.show()

#Normalization
Xnorm, Ynorm = myML.norm0to1_minmax([1,1],[0,0]), myML.norm0to1_minmax([14],[0])
#X_train, Y_train = Xnorm.normalize(X_train), Ynorm.normalize(Y_train)
Y_train = Ynorm.normalize(Y_train)

In [ ]:
Ynorm.recover(Y_train)

In [ ]:
#Cross-validation using the training samples to find the elbow of the curve (to avoid under or overfitting)
epochs = 500        #Times that the entire dataset is used to train the NN
lr = 2            #Learning rate
lambd = 0           #Regularization term
activation = ['ReLU','sigmoid']
lossname = 'MSE'
optimizer = 'GD'

n = 14
yplot_train = np.zeros(n)
times = 10
for i in range(1,n+1):
    nodes = [X_train.shape[0], i, Y_train.shape[0]]
    #print('Trying with '+str(i)+' nodes in the hidden layer')
    aux_train = 500
    for j in range(times):
        np.random.seed(None)
        neuralnetwork = myML.ANN(nodes, activation, lossname)
        costs = neuralnetwork.train(X_train, Y_train, epochs, optimizer, lr, lambd)
        if costs[epochs-1]<aux_train: aux_train = costs[epochs-1]

    yplot_train[i-1] = aux_train

plt.plot(range(1,n+1),yplot_train,label='Training set error')
plt.xlabel('Nº of neurons in hidden layer')
plt.ylabel('Cost function error')
plt.legend(loc='best')
plt.show()

In [ ]:
#Train a model for future use
nodes = [X_train.shape[0], 6, Y_train.shape[0]]
activation = ['ReLU','sigmoid']
loss = 'MSE'
epochs = 10000

model = myML.ANN(nodes, activation, loss)
costs = model.train(X_train, Y_train, epochs, 'GD', lr=0.1, lambd=0, printepochs=False)

print(Ynorm.recover(Y_train))
print(Ynorm.recover(model.test(X_train)))

plt.plot(range(epochs),costs)
plt.xlabel('Epoch')
plt.ylabel('Cost function error')
plt.show()
costs[-1]

In [ ]:
# Load training images. Each image shouldn't have unwanted pixels (surroundings) and must be named with the known pH value
location = os.getcwd()+'\\test\\'
X = np.empty((2,1))

for count, file in enumerate(os.listdir(location)):
    RGB = [np.mean(x) for x in np.array(Image.open(location+file)).T]
    X[:,0] = RGB[0]/(RGB[0]+RGB[1]+RGB[2]), RGB[1]/(RGB[0]+RGB[1]+RGB[2])
    print(file.split('.jpg')[0])
    X = Xnorm.normalize(X)

    Y = model.test(X)
    print(round(Ynorm.recover(Y)[0,0],3))